# Creating a Tool Calling Agent and an Agent Executor

In [1]:
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools import WikipediaQueryRun
from langchain_chroma import Chroma

from langchain_huggingface import HuggingFaceEmbeddings
from langchain_ollama import ChatOllama

from langchain.tools.retriever import create_retriever_tool
from langchain.tools import tool

from langchain import hub

from platform import python_version

from langchain.agents import create_tool_calling_agent, AgentExecutor

In [2]:
wikipedia_tool = WikipediaQueryRun(api_wrapper = WikipediaAPIWrapper())

In [3]:
vectorstore = Chroma(persist_directory = "./intro-to-ds-lectures", 
                     embedding_function = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2"))

retriever = vectorstore.as_retriever(search_type = 'mmr', 
                                     search_kwargs = {'k':3, 
                                                      'lambda_mult':0.7})

retriever_tool = create_retriever_tool(retriever = retriever, 
                                       name = "Introduction-to-Data-and-Data-Science-Course-Lectures", 
                                       description = '''For any questions regarding 
                                       the Introduction to Data and Data Science course, you must use this tool.''')

In [4]:
@tool
def get_python_version() -> str:
    ''' Useful for questions regarding the version of Python currently used. '''
    return python_version()

In [5]:
tools = [wikipedia_tool, retriever_tool, get_python_version]

In [6]:
chat = ChatOllama(model="llama3.2", 
                  temperature=0,
                  seed=365)

In [7]:
chat_prompt_template = hub.pull("hwchase17/openai-tools-agent")

c:\Users\Eda AYDIN\AppData\Local\Programs\Python\Python312\Lib\site-packages\langsmith\client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [8]:
agent = create_tool_calling_agent(llm = chat, 
                                  tools = tools, 
                                  prompt = chat_prompt_template)

In [9]:
agent_executor = AgentExecutor(agent = agent, 
                               tools = tools, 
                               verbose = True, 
                               return_intermediate_steps = True)

In [10]:
response = agent_executor.invoke({"input":"Could you tell me the version of Python I'm currently using?"})



> Entering new AgentExecutor chain...

Invoking: `get_python_version` with `{}`


3.12.6The current version of Python is 3.12.6.

> Finished chain.


In [11]:
response

{'input': "Could you tell me the version of Python I'm currently using?",
 'output': 'The current version of Python is 3.12.6.',
 'intermediate_steps': [(ToolAgentAction(tool='get_python_version', tool_input={}, log='\nInvoking: `get_python_version` with `{}`\n\n\n', message_log=[AIMessageChunk(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-05-23T18:33:24.7777539Z', 'done': True, 'done_reason': 'stop', 'total_duration': 440780200, 'load_duration': 49753500, 'prompt_eval_count': 326, 'prompt_eval_duration': 42048600, 'eval_count': 12, 'eval_duration': 346356700, 'model_name': 'llama3.2'}, id='run--8c580090-5d4e-4f2e-b162-2b723923fcf4', tool_calls=[{'name': 'get_python_version', 'args': {}, 'id': '937b1c1a-93a1-4d75-8da6-1decbcf0aecf', 'type': 'tool_call'}], usage_metadata={'input_tokens': 326, 'output_tokens': 12, 'total_tokens': 338}, tool_call_chunks=[{'name': 'get_python_version', 'args': '{}', 'id': '937b1c1a-93a1-4d75-8da6-1decbcf0aecf

In [12]:
response = agent_executor.invoke({"input":'''
I’m currently following the Introduction to Data and Data Science course. 
Could you list the programming languages a data scientist should know? 
Additionally, could you tell me who their creators are?
'''})



> Entering new AgentExecutor chain...

Invoking: `Introduction-to-Data-and-Data-Science-Course-Lectures` with `{'query': 'programming languages for data scientists and their creators'}`


The programming languages that a data scientist should know are:

1. Python: Created by Guido van Rossum in 1991, Python is one of the most popular and widely used programming languages in data science.
2. R: Developed by Ross Ihaka and Robert Gentleman in 1993, R is a popular language for statistical computing and data visualization.
3. SQL: Created by Donald Chamberlin and Raymond Boyce in 1974, SQL (Structured Query Language) is a standard language for managing relational databases.
4. Julia: Developed by Jeff Bezanson, Alan Edelman, Brian Smith, and Stefan Karpinski in 2012, Julia is a new language that aims to provide high performance and dynamism for data science applications.

These languages are commonly used in data science for tasks such as data cleaning, visualization, modeling, and machi

In [13]:
response

{'input': '\nI’m currently following the Introduction to Data and Data Science course. \nCould you list the programming languages a data scientist should know? \nAdditionally, could you tell me who their creators are?\n',
 'output': 'The programming languages that a data scientist should know are:\n\n1. Python: Created by Guido van Rossum in 1991, Python is one of the most popular and widely used programming languages in data science.\n2. R: Developed by Ross Ihaka and Robert Gentleman in 1993, R is a popular language for statistical computing and data visualization.\n3. SQL: Created by Donald Chamberlin and Raymond Boyce in 1974, SQL (Structured Query Language) is a standard language for managing relational databases.\n4. Julia: Developed by Jeff Bezanson, Alan Edelman, Brian Smith, and Stefan Karpinski in 2012, Julia is a new language that aims to provide high performance and dynamism for data science applications.\n\nThese languages are commonly used in data science for tasks such a